In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
model_category = 'Women Tops & Tunics'

In [ ]:
path_eff_b6 = "../../Models/eff_net_b6_fillna_eff_net_Women Tops Tunics.pth"
path_swin_v2 = "../../Models/Tops_swin_v2.pth"
path_mobile_v3_large = "../../Models/mobile_net_v3_large_fillna_eff_net_Women Tops_Tunics.pth"

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Women Tops Tunics_effnet_b5_4epochs.csv')
train_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
0,51375,Women Tops & Tunics,10,black,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/051...
1,51376,Women Tops & Tunics,10,navy blue,fitted,crop,high,casual,default,solid,short sleeves,default,knitted,/kaggle/input/visual-taxonomy/train_images/051...
2,51377,Women Tops & Tunics,10,red,regular,regular,round neck,casual,printed,typography,sleeveless,sleeveless,knitted,/kaggle/input/visual-taxonomy/train_images/051...
3,51378,Women Tops & Tunics,10,default,fitted,crop,stylised,casual,solid,solid,short sleeves,regular sleeves,default,/kaggle/input/visual-taxonomy/train_images/051...
4,51379,Women Tops & Tunics,10,default,boxy,regular,round neck,casual,printed,typography,short sleeves,default,tie-ups,/kaggle/input/visual-taxonomy/train_images/051...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18999,70374,Women Tops & Tunics,10,multicolor,fitted,regular,square neck,casual,printed,default,short sleeves,regular sleeves,ruffles,/kaggle/input/visual-taxonomy/train_images/070...
19000,70375,Women Tops & Tunics,10,yellow,regular,crop,round neck,casual,default,default,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...
19001,70376,Women Tops & Tunics,10,maroon,fitted,crop,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted,/kaggle/input/visual-taxonomy/train_images/070...
19002,70377,Women Tops & Tunics,10,green,regular,regular,high,party,solid,solid,short sleeves,default,ruffles,/kaggle/input/visual-taxonomy/train_images/070...


In [6]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']


In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [10]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel1(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel1, self).__init__()
        # Use MobileNetV2 as the base model
        self.base_model = models.efficientnet_b6(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove original classification layer
        
        # Add an adaptive pooling layer to make sure output is flat
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D
        
        # Dynamically create a fully connected layer for each attribute
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(2304, n_classes)  # Adjust input to 1280 for MobileNetV2

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features
        x = self.pooling(x)  # Adaptive pool to (1, 1) shape
        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)
        
        outputs = {}
        # Dynamically compute output for each attribute
        for attr, layer in self.output_layers.items():
            outputs[attr] = layer(x)
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model_eff_b6 = MultiOutputModel1(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_eff_b6.parameters(), lr=0.0001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B6_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B6_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b6_lukemelas-24a108a5.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b6_lukemelas-24a108a5.pth
100%|██████████| 165M/165M [00:02<00:00, 60.8MB/s] 


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel2(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel2, self).__init__()
        # Use MobileNetV2 as the base model
        # Load a pretrained Swin V2 model
        self.base_model = models.swin_v2_b(weights="IMAGENET1K_V1")
        
        # Remove the original classification head
        in_features = self.base_model.head.in_features  # Get the input features for the head
        self.base_model.head = nn.Identity()  # Replace with identity to extract embeddings
        
        # Define output layers for each attribute dynamically
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(in_features, n_classes)  # Use `in_features` for each linear layer

    def forward(self, x):
        # Pass input through the Swin model to get embeddings
        x = self.base_model(x)
        
        # Compute output for each attribute
        outputs = {attr: layer(x) for attr, layer in self.output_layers.items()}
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model_swin_v2 = MultiOutputModel2(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_swin_v2.parameters(), lr=0.0001)

Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:01<00:00, 185MB/s]  


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModel3(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModel3, self).__init__()
        # Use MobileNetV2 as the base model
        self.base_model = models.mobilenet_v3_large(pretrained=True)
        self.base_model.classifier = nn.Identity()  # Remove original classification layer
        
        # Add an adaptive pooling layer to make sure output is flat
        self.pooling = nn.AdaptiveAvgPool2d((1, 1))  # Convert 2D output to 1D
        
        # Dynamically create a fully connected layer for each attribute
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(960, n_classes)  # Adjust input to 1280 for MobileNetV2

    def forward(self, x):
        x = self.base_model.features(x)  # Extract features
        x = self.pooling(x)  # Adaptive pool to (1, 1) shape
        x = torch.flatten(x, 1)  # Flatten the output to (batch_size, 1280)
        
        outputs = {}
        # Dynamically compute output for each attribute
        for attr, layer in self.output_layers.items():
            outputs[attr] = layer(x)
        
        return list(outputs.values())

# Example usage
num_classes = {}
for key in attr_columns:
    num_classes[key] = train_df_Men_Tshirts[key].nunique()

model_mobile_net_v3_large = MultiOutputModel3(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mobile_net_v3_large.parameters(), lr=0.0001)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 124MB/s] 


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_eff_b6 = model_eff_b6.to(device)
model_swin_v2 =  model_swin_v2.to(device)
model_mobile_net_v3_large = model_mobile_net_v3_large.to(device)
# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path=f"eff_net_b6_fillna_eff_net_{model_category}.pth"):
    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)
    best_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
                outputs = model(images)
                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")

# Run training
# train(model, train_loader, val_loader, criterion, optimizer, num_epochs=7)


In [15]:
torch.manual_seed(42)
model_eff_b6.load_state_dict(torch.load(f"{path_eff_b6}"))

/tmp/ipykernel_30/563325272.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_eff_b6.load_state_dict(torch.load(f"{path_eff_b6}"))


<All keys matched successfully>

In [16]:
torch.manual_seed(42)
model_swin_v2.load_state_dict(torch.load(f"{path_swin_v2}"))

/tmp/ipykernel_30/3335458680.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_swin_v2.load_state_dict(torch.load(f"{path_swin_v2}"))


<All keys matched successfully>

In [17]:
torch.manual_seed(42)
model_mobile_net_v3_large.load_state_dict(torch.load(f"{path_mobile_v3_large}"))

/tmp/ipykernel_30/2865226030.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_mobile_net_v3_large.load_state_dict(torch.load(f"{path_mobile_v3_large}"))


<All keys matched successfully>

In [18]:
model_eff_b6.eval()

MultiOutputModel1(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 56, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(56, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=56, bias=False)
              (1): BatchNorm2d(56, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(56, 14, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(14, 56, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
         

In [19]:
model_swin_v2.eval()

MultiOutputModel2(
  (base_model): SwinTransformer(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (1): Permute()
        (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): SwinTransformerBlockV2(
          (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (attn): ShiftedWindowAttentionV2(
            (qkv): Linear(in_features=128, out_features=384, bias=True)
            (proj): Linear(in_features=128, out_features=128, bias=True)
            (cpb_mlp): Sequential(
              (0): Linear(in_features=2, out_features=512, bias=True)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=512, out_features=4, bias=False)
            )
          )
          (stochastic_depth): StochasticDepth(p=0.0, mode=row)
          (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (0):

In [20]:
model_mobile_net_v3_large.eval()

MultiOutputModel3(
  (base_model): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          )
        )
      )
      (2): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNorm

In [ ]:
test_df = pd.read_csv('../../Dataset/test.csv')
test_df

,id,Category
0,0,Men Tshirts
1,1,Men Tshirts
2,2,Men Tshirts
3,3,Men Tshirts
4,4,Men Tshirts
...,...,...
30200,30484,Women Tops & Tunics
30201,30485,Women Tops & Tunics
30202,30486,Women Tops & Tunics
30203,30487,Women Tops & Tunics


In [ ]:
test_df_Men_Tshirts = test_df[test_df['Category'] == model_category]
test_df_Men_Tshirts['len'] = 10

def format_image_path_test(row):
    return f"../../Dataset/test_images/{str(row['id']).zfill(6)}.jpg"

test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)
test_df_Men_Tshirts

/tmp/ipykernel_30/2828864867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['len'] = 10
/tmp/ipykernel_30/2828864867.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)


,id,Category,len,image_path
23280,23564,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...
23281,23565,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...
23282,23566,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...
23283,23567,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...
23284,23568,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...
...,...,...,...,...
30200,30484,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...
30201,30485,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...
30202,30486,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...
30203,30487,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...


In [23]:
# test_df_Men_Tshirts = test_df_Men_Tshirts.sample(128)

In [24]:
# Test dataset without labels
torch.manual_seed(42)
test_dataset = CustomDataset(dataframe=test_df_Men_Tshirts, img_dir='', transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [25]:
len(test_loader)

217

In [26]:
from tqdm import tqdm  # Use console version of tqdm

predictions = []

with torch.no_grad():
    for images in tqdm(test_loader):  # This will display in the console
        images = images.cuda() if torch.cuda.is_available() else images

        # Forward pass through both models
        torch.manual_seed(42)
        outputs1 = model_eff_b6(images)
        outputs2 = model_swin_v2(images)
        outputs3 = model_mobile_net_v3_large(images)
#         outputs3 = model_eff_b5(images)

        # Initialize a list to hold blended predictions for the batch
        batch_preds = []

        # Loop through the outputs and blend predictions for each attribute
        for out1, out2 ,out3 in zip(outputs1, outputs2,outputs3):
            # Blend logits by averaging
            torch.manual_seed(42)
            blended_output = (out1 + out2 + out3)/3
            # Get the predicted classes from the blended output
            _, pred = torch.max(blended_output, 1)
            batch_preds.append(pred.cpu().numpy())  # Move to CPU and convert to numpy

        # Stack predictions along a new dimension and add to predictions list
        predictions.append(np.stack(batch_preds, axis=1))  # Shape: (batch_size, num_attributes)

# Combine all predictions into a single array
predictions = np.concatenate(predictions, axis=0)

# Display final predictions
print(predictions)  # This will be an array with shape (num_samples, num_attributes)


100%|██████████| 217/217 [04:25<00:00,  1.23s/it]

[[ 6  2  0 ...  2  3  2]
 [11  0  0 ...  1  2  1]
 [ 0  2  1 ...  0  2  0]
 ...
 [ 2  3  1 ...  1  0  2]
 [ 5  3  1 ...  0  2  3]
 [ 8  2  0 ...  2  0  2]]


In [27]:
# Assuming predictions is a numpy array of shape (N, 5)
# Add new columns attr_1 to attr_10 to test_df
for i in range(1, 11):
    test_df_Men_Tshirts[f'attr_{i}'] = np.nan 

# Assign predictions to attr_1 to attr_5
test_df_Men_Tshirts[attr_columns] = predictions

# Optionally save the updated test_df to CSV
# test_df.to_csv('test_predictions_with_attrs.csv', index=False)

# Display the first few rows of the updated DataFrame
test_df_Men_Tshirts.head()

/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[f'attr_{i}'] = np.nan
/tmp/ipykernel_30/1201814413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,6,2,0,1,0,0,0,2,3,2
23281,23565,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,11,0,0,2,0,1,2,1,2,1
23282,23566,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,0,2,1,2,0,2,4,0,2,0
23283,23567,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,8,3,1,6,0,2,4,1,0,2
23284,23568,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,0,2,1,2,0,0,4,1,2,2


In [28]:
# Inverse transform predictions for each attribute using the stored label encoders
for attr in attr_columns:
    # Inverse transform using the corresponding label encoder
    test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))

# Check the updated DataFrame¯ with original attribute names instead of encoded numbers
test_df_Men_Tshirts

/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))
/tmp/ipykernel_30/688606145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,navy blue,fitted,crop,high,casual,default,default,sleeveless,sleeveless,knitted
23281,23565,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,yellow,boxy,crop,round neck,casual,printed,graphic,short sleeves,regular sleeves,default
23282,23566,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,black,fitted,regular,round neck,casual,solid,solid,long sleeves,regular sleeves,applique
23283,23567,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,pink,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
23284,23568,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0235...,black,fitted,regular,round neck,casual,default,solid,short sleeves,regular sleeves,knitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30484,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...,green,boxy,crop,round neck,casual,printed,typography,short sleeves,regular sleeves,tie-ups
30201,30485,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...,default,regular,regular,round neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30202,30486,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...,default,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
30203,30487,Women Tops & Tunics,10,/kaggle/input/visual-taxonomy/test_images/0304...,multicolor,regular,regular,round neck,casual,default,default,long sleeves,regular sleeves,ruffles


In [29]:
# Get value counts for each specified column
columns_of_interest = attr_columns

for column in columns_of_interest:
    print(f"Value counts for {column}:\n")
    print(test_df_Men_Tshirts[column].value_counts(dropna=False))  # Including NaN values
    print("\n" + "-"*50 + "\n")

Value counts for attr_1:

attr_1
black         903
white         834
pink          766
default       725
blue          627
yellow        614
red           599
maroon        569
green         502
navy blue     291
peach         264
multicolor    231
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_2:

attr_2
fitted     2952
regular    2527
boxy        778
default     668
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_3:

attr_3
regular    4407
crop       2518
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_4:

attr_4
round neck         2847
square neck         886
high                739
sweetheart neck     701
default             684
v-neck              660
stylised            408
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_5:

attr_5
casual    6738
party      187
Name: cou

In [ ]:
sub_df = pd.read_csv('../../Dataset/sample_submission.csv')
sub_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,47001,Women Tshirts,8,maroon,regular,crop,printed,typography,short sleeves,regular sleeves,default,nu,nu
1,16431,Sarees,10,same as saree\n,temple border,small border,cream,party,jacquard,woven design,zari woven,ethnic motif,no
2,55700,Women Tops & Tunics,10,white,fitted,regular,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted
3,15698,Sarees,10,same as saree\n,zari,small border,white,party,jacquard,woven design\n,zari woven,floral,no
4,30330,Kurtis,9,yellow,a-line,knee length\n,daily,net,default,solid,three-quarter sleeves,regular,nu


In [31]:
test_df_Men_Tshirts = test_df_Men_Tshirts.fillna('dummy')

In [ ]:
sub_df_Men_Tshirts = test_df_Men_Tshirts.drop('image_path', axis = 1)
# sub_df_Men_Tshirts.replace("unknown", np.nan, inplace=True)
sub_df_Men_Tshirts.to_csv(f"output/sub_df_{model_category}_effnet_0.33_b6_0.33_swin_v2_0.33_mobile_net_v3_large_blending.csv", index = False)
sub_df_Men_Tshirts.to_csv(f"sub_df_{model_category}_effnet_0.33_b6_0.33_swin_v2_0.33_mobile_net_v3_large_blending.csv", index = False)

sub_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,10,navy blue,fitted,crop,high,casual,default,default,sleeveless,sleeveless,knitted
23281,23565,Women Tops & Tunics,10,yellow,boxy,crop,round neck,casual,printed,graphic,short sleeves,regular sleeves,default
23282,23566,Women Tops & Tunics,10,black,fitted,regular,round neck,casual,solid,solid,long sleeves,regular sleeves,applique
23283,23567,Women Tops & Tunics,10,pink,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
23284,23568,Women Tops & Tunics,10,black,fitted,regular,round neck,casual,default,solid,short sleeves,regular sleeves,knitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30484,Women Tops & Tunics,10,green,boxy,crop,round neck,casual,printed,typography,short sleeves,regular sleeves,tie-ups
30201,30485,Women Tops & Tunics,10,default,regular,regular,round neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30202,30486,Women Tops & Tunics,10,default,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
30203,30487,Women Tops & Tunics,10,multicolor,regular,regular,round neck,casual,default,default,long sleeves,regular sleeves,ruffles
